In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor



general:

In [ ]:
data = pd.read_csv("DS_Proyecto_01_Datos_Properati.csv")

data.head()

,start_date,end_date,created_on,lat,lon,l1,l2,l3,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,title,description,property_type,operation_type
0,2019-10-17,2019-12-23,2019-10-17,-34.605880,-58.384949,Argentina,Capital Federal,San Cristobal,7.0,7.0,2.0,140.0,140.0,153000.0,USD,"***Venta semipiso centro, ideal hostel*****",DESCRIPCION DE LA PROPIEDAD: Departamento de 1...,Departamento,Venta
1,2019-10-17,2019-11-21,2019-10-17,-34.624056,-58.412110,Argentina,Capital Federal,Boedo,2.0,1.0,2.0,70.0,58.0,159000.0,USD,Espectacular PH reciclado en Boedo sin expensas.,PH reciclado en Boedo a una cuadra de la plaz...,PH,Venta
2,2019-10-17,2019-11-01,2019-10-17,-34.593569,-58.427474,Argentina,Capital Federal,Palermo,2.0,1.0,1.0,45.0,45.0,125000.0,USD,Depto.tipo casa de 2 ambientes en Venta en Pal...,"2 ambienets amplio , excelente estado , patio ...",PH,Venta
3,2019-10-17,2019-12-23,2019-10-17,-34.581294,-58.436754,Argentina,Capital Federal,Palermo,2.0,1.0,1.0,85.0,50.0,295000.0,USD,COSTA RICA 5800 / PALERMO HOLLYWOOD / VENTA PH...,HERMOSO PH EN PALERMO!!!2 AMBIENTES TOTALMENTE...,PH,Venta
4,2019-10-17,2020-03-11,2019-10-17,-34.914194,-57.938219,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,2.0,1.0,1.0,50.0,35.0,40000.0,USD,58 entre 1 y 2 Venta de departamento en ph.1 ...,58 entre 1 y 2 Venta de departamento en PH. P...,PH,Venta


In [ ]:
data.shape

(146660, 19)

In [ ]:
data = data.drop_duplicates()

In [ ]:
lista_palabras = ['jardin','cochera','parrilla','pileta','terraza','lavadero']
#creamos nueva columna con palabras de descripción en una lista
data['desc_lista']=[x.lower().split(' ') for x in data['description']]
#creamos nuevos atributos suponiendo que si la palabra aparece en la descripcion entonces la propiedad cuenta con esa caracteristica
for palabra in lista_palabras:
  data[[palabra]] = data.apply(
    lambda row: sum(i == palabra for i in row['desc_lista'])>=1,
    axis=1)
#convertimos bool a int
for palabra in lista_palabras:
  data[[palabra]] = data[[palabra]].astype(int)

#chequeamos cantidades de propiedades para cada palabra
for palabra in lista_palabras:
    print(palabra, data[palabra].sum())

jardin 3065
cochera 27573
parrilla 17787
pileta 12769
terraza 23475
lavadero 31902


In [ ]:
#Descartamos la columna auxiliar creada
data = data.drop(['desc_lista'],axis = 1)

data = data.drop(['start_date','end_date','created_on', 'l1', 'currency', 'operation_type','title','description'], axis =1)
data = data[data.property_type.isin(['Departamento', 'Casa', 'PH'])] 



outliers:

In [ ]:
nuemerical_cols = ['lat', 'lon', 'rooms','bedrooms', 'bathrooms', 'surface_total', 'surface_covered', 'price']
# Método 3 sigmas - estudiamos intervalos
dictio_3s = {}
for columna in nuemerical_cols:
  #tres sigmas
  #print(str(columna))
  columna_mean = data[columna].mean()
  columna_sdt = data[columna].std() 
  
  min_3s = columna_mean  - 3*columna_sdt
  max_3s = columna_mean  + 3*columna_sdt
  dictio_3s[columna] = [min_3s,max_3s]
  #print('(' + str(min_3s) + ',' + str(max_3s) + ')')
  mascara_outliers = np.logical_or(data[columna] < min_3s, data[columna] > max_3s)

dictio_IQR = {}
for columna in nuemerical_cols:
  Q1 = data[columna].quantile(0.25)
  Q3 = data[columna].quantile(0.75)
  IQR = Q3-Q1
  min_IQR = Q1 - 3/2*IQR
  max_IQR = Q3 + 3/2*IQR
  dictio_IQR[columna] = [min_IQR, max_IQR]
  mascara_outliers = np.logical_or(data[columna] < min_IQR, data[columna] > max_IQR)



In [ ]:
dictio_3s

{'bathrooms': [-1.0611251110001063, 4.254699466972003],
 'bedrooms': [-1.3935497115587987, 5.396404379153351],
 'lat': [-36.03377373479614, -33.17372612892605],
 'lon': [-59.94839914775234, -56.980686819429565],
 'price': [-620709.6925778633, 1095138.600347956],
 'rooms': [-1.0914895769670951, 7.291515141154509],
 'surface_covered': [-2667.1765244333283, 2893.0192431342184],
 'surface_total': [-4162.348446703714, 4529.838504113495]}

In [ ]:
# Creamos masacaras y filtramos en cada atributo elegimos el método más conveniente 
mascara_outliers = np.logical_and(
np.logical_not(np.logical_or(data['lat'] < dictio_3s['lat'][0]  , data['lat'] > dictio_3s['lat'][1])),
np.logical_not(np.logical_or(data['lon'] < dictio_3s['lon'][0]  , data['lon'] > dictio_3s['lon'][1])))
print(mascara_outliers.sum())
data = data[mascara_outliers]
print(data.shape)

mascara_outliers1 = np.logical_and(
np.logical_not(np.logical_or(data['rooms'] < dictio_3s['rooms'][0]  , data['rooms'] > dictio_3s['rooms'][1])),
np.logical_not(np.logical_or(data['bedrooms'] < dictio_3s['bedrooms'][0]  , data['bedrooms'] > dictio_3s['bedrooms'][1])))
print(mascara_outliers1.sum())
data = data[mascara_outliers1]
print(data.shape)

mascara_outliers2 = np.logical_and(
np.logical_not(np.logical_or(data['bathrooms'] < dictio_IQR['bathrooms'][0]  , data['bathrooms'] > dictio_IQR['bathrooms'][1])),
np.logical_not(np.logical_or(data['price'] < dictio_3s['price'][0]  , data['price'] > dictio_3s['price'][1])))
print(mascara_outliers2.sum())
data = data[mascara_outliers2]
print(data.shape)

mascara_outliers3 = np.logical_and(
np.logical_not(np.logical_or(data['surface_total'] < dictio_IQR['surface_total'][0]  , data['surface_total'] > dictio_IQR['surface_total'][1])),
np.logical_not(np.logical_or(data['surface_covered'] < dictio_IQR['surface_covered'][0]  , data['surface_covered'] > dictio_IQR['surface_covered'][1])))
print(mascara_outliers3.sum())
data = data[mascara_outliers3]
print(data.shape)


140818
(140818, 17)
138920
(138920, 17)
133282
(133282, 17)
122318
(122318, 17)


In [ ]:
#Filtramos a ojo las siguientes:
data = data[(data.price>=10000) & (data.bedrooms>=1)]

#Filtramos a ojo las siguientes:
data = data[(data.surface_total>=data.surface_covered) & (data.surface_total>=15)]

In [ ]:
data = pd.get_dummies(data, columns=['property_type','l2'])

faltantes:

In [ ]:
data['bathrooms'] = data.apply(
    lambda row: (row['rooms']//2 +1) if np.isnan(row['bathrooms']) else row['bathrooms'],
    axis=1
)

#Completamos faltantes en lat y lon usando el promedio de los valores agrupando por barrio respectivamente.
data['lat'].fillna(data.groupby(['l3'])['lat'].transform('mean'), inplace=True)
data['lon'].fillna(data.groupby(['l3'])['lon'].transform('mean'), inplace=True)

data.isnull().sum()





lat                            0
lon                            0
l3                             0
rooms                          0
bedrooms                       0
bathrooms                      0
surface_total                  0
surface_covered                0
price                          0
jardin                         0
cochera                        0
parrilla                       0
pileta                         0
terraza                        0
lavadero                       0
property_type_Casa             0
property_type_Departamento     0
property_type_PH               0
l2_Bs.As. G.B.A. Zona Norte    0
l2_Bs.As. G.B.A. Zona Oeste    0
l2_Bs.As. G.B.A. Zona Sur      0
l2_Capital Federal             0
dtype: int64

#MODELO
XGBoost

In [ ]:
#Aplicamos logaritmo a nuestra columna target
data = data.drop(['l3'], axis=1)
data['price'] = np.log(data['price'])
data.head()

,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,jardin,cochera,parrilla,pileta,terraza,lavadero,property_type_Casa,property_type_Departamento,property_type_PH,l2_Bs.As. G.B.A. Zona Norte,l2_Bs.As. G.B.A. Zona Oeste,l2_Bs.As. G.B.A. Zona Sur,l2_Capital Federal
1,-34.624056,-58.412110,2.0,1.0,2.0,70.0,58.0,11.976659,0,0,0,0,0,0,0,0,1,0,0,0,1
2,-34.593569,-58.427474,2.0,1.0,1.0,45.0,45.0,11.736069,0,0,0,0,0,0,0,0,1,0,0,0,1
3,-34.581294,-58.436754,2.0,1.0,1.0,85.0,50.0,12.594731,0,0,0,0,0,0,0,0,1,0,0,0,1
4,-34.914194,-57.938219,2.0,1.0,1.0,50.0,35.0,10.596635,0,0,0,0,1,0,0,0,1,0,0,1,0
5,-34.593779,-58.441239,2.0,1.0,1.0,56.0,56.0,11.918391,0,0,0,0,1,0,0,0,1,0,0,0,1


In [ ]:
data.columns

Index(['lat', 'lon', 'rooms', 'bedrooms', 'bathrooms', 'surface_total',
       'surface_covered', 'price', 'jardin', 'cochera', 'parrilla', 'pileta',
       'terraza', 'lavadero', 'property_type_Casa',
       'property_type_Departamento', 'property_type_PH',
       'l2_Bs.As. G.B.A. Zona Norte', 'l2_Bs.As. G.B.A. Zona Oeste',
       'l2_Bs.As. G.B.A. Zona Sur', 'l2_Capital Federal'],
      dtype='object')

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
X_train_xgb1, X_test_xgb1 = train_test_split(data, test_size=.3, random_state=1999)

X_train_xgb = xgb.DMatrix(X_train_xgb1.drop(['price'], axis= 1), label=X_train_xgb1['price'])
X_test_xgb = xgb.DMatrix(X_test_xgb1.drop(['price'], axis = 1), label=X_test_xgb1['price'])

parametros = {"booster":"gbtree", "max_depth": 15, "eta": 0.3, "nthread":3}
rondas = 20

modelo = xgb.train(parametros, X_train_xgb, rondas)

prediccion = modelo.predict(X_test_xgb)
prediccion_train = modelo.predict(X_train_xgb)
expy_train = np.exp(X_train_xgb1['price'])
expy_train_pred = np.exp(prediccion_train)
expy_test = np.exp(X_test_xgb1['price'])
expy_test_pred = np.exp(prediccion)


In [ ]:
modelo.save_model('0001.model')
# dump model
modelo.dump_model('dump.raw.txt')
# dump model with feature map
# modelo.dump_model('dump.raw.txt', 'featmap.txt')

In [ ]:
filename = 'modelo.sav'
pickle.dump(modelo, open(filename, 'wb'))

In [ ]:
rmse_train = np.sqrt(mean_squared_error(np.exp(X_train_xgb1['price']), np.exp(prediccion_train)))
mae_train = mean_absolute_error(np.exp(X_train_xgb1['price']), np.exp(prediccion_train))
r2_train = r2_score(np.exp(X_train_xgb1['price']), np.exp(prediccion_train))
print(str(modelo).upper())
print("ERROR  TRAIN    ")
print("------ ---------")
print("%-7s %-9.2f  "%('RECM:',rmse_train))
print("%-7s %-9.2f  "%('MAE:',mae_train))
print("%-10s %-9.2f  "%('R2:',r2_train))

rmse_test = np.sqrt(mean_squared_error(np.exp(X_test_xgb1['price']), np.exp(prediccion)))
mae_test = mean_absolute_error(np.exp(X_test_xgb1['price']), np.exp(prediccion))
r2_test = r2_score(np.exp(X_test_xgb1['price']), np.exp(prediccion))
print(str(modelo).upper())
print("ERROR  TEST    ")
print("------ --------")
print("%-6s %-9.2f  "%('RECM:',rmse_test))
print("%-6s %-9.2f  "%('MAE:',mae_test))
print("%-10s %-9.2f  "%('R2:',r2_test))

<XGBOOST.CORE.BOOSTER OBJECT AT 0X7F0D0B0C4310>
ERROR  TRAIN    
------ ---------
RECM:   29348.78   
MAE:    17043.86   
R2:        0.95       
<XGBOOST.CORE.BOOSTER OBJECT AT 0X7F0D0B0C4310>
ERROR  TEST    
------ --------
RECM:  41877.53   
MAE:   23899.08   
R2:        0.89       


prueba con cuba:

In [ ]:
ejemplo = [-34.5588753, -58.4563418, 3, 2, 2, 90, 80, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1]

In [ ]:
df = pd.DataFrame(columns=['lat', 'lon', 'rooms', 'bedrooms', 'bathrooms', 'surface_total',
       'surface_covered', 'jardin', 'cochera', 'parrilla', 'pileta',
       'terraza', 'lavadero', 'property_type_Casa',
       'property_type_Departamento', 'property_type_PH',
       'l2_Bs.As. G.B.A. Zona Norte', 'l2_Bs.As. G.B.A. Zona Oeste',
       'l2_Bs.As. G.B.A. Zona Sur', 'l2_Capital Federal'])

In [ ]:
df.loc[0] = ejemplo

In [ ]:
df

,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,jardin,cochera,parrilla,pileta,terraza,lavadero,property_type_Casa,property_type_Departamento,property_type_PH,l2_Bs.As. G.B.A. Zona Norte,l2_Bs.As. G.B.A. Zona Oeste,l2_Bs.As. G.B.A. Zona Sur,l2_Capital Federal
0,-34.558875,-58.456342,3.0,2.0,2.0,90.0,80.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
modelo.predict(xgb.DMatrix(df))

array([12.91988], dtype=float32)

In [ ]:
int(np.exp(modelo.predict(xgb.DMatrix(df))))

408349

Modelo con fiteado con todo el dataset:

In [ ]:
X_xgb = xgb.DMatrix(data.drop(['price'], axis= 1), label=data['price'])
parametros = {"booster":"gbtree", "max_depth": 15, "eta": 0.3, "nthread":3}
rondas = 20
model = xgb.train(parametros, X_xgb, rondas)
#Este habria que exportar!

In [ ]:
int(np.exp(model.predict(xgb.DMatrix(df))))

413452

In [ ]:
import pickle

In [ ]:
# save the model to disk
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
int(np.exp(loaded_model.predict(xgb.DMatrix(df))))


413452

In [ ]:
a, b ,c = [1,2,3]
print(c)

3
